In [1]:
import pandas as pd

In [2]:
df_tramites_proc_X = pd.read_parquet('./data/tramita/tramites_autoconsumo.parquet')

In [3]:
df_tramites_proc_X.head()

,id_exp,dni,nif,fecha_alta_exp,fecha_registro_exp,codine_provincia,codine_municipio,codine,municipio,provincia,...,siaci,consejeria_proc,org_instructor_proc,es_telematica,desc_tramite,fecha_tramite,num_tramite,cod_procedimiento,orden_tramite,es_inicial
45875,1127895,1787973,<NA>,2022-02-09 09:07:17,2021-12-23,13,064,13064,Poblete,Ciudad Real,...,SLPT,Consejeria de desarrollo sostenible,Direccion general de transicion energetica,True,Registro de solicitud,2021-12-23 00:00:00,0,884.0,0,False
45876,1127895,1787973,<NA>,2022-02-09 09:07:17,2021-12-23,13,064,13064,Poblete,Ciudad Real,...,SLPT,Consejeria de desarrollo sostenible,Direccion general de transicion energetica,True,1-1-Presentación de la solicitud,2022-02-09 09:07:17,1,884.0,1,True
45877,1127895,1787973,<NA>,2022-02-09 09:07:17,2021-12-23,13,064,13064,Poblete,Ciudad Real,...,SLPT,Consejeria de desarrollo sostenible,Direccion general de transicion energetica,True,10-1-Requerimiento documentación,2022-10-14 09:44:23,10,884.0,2,False
45878,1127895,1787973,<NA>,2022-02-09 09:07:17,2021-12-23,13,064,13064,Poblete,Ciudad Real,...,SLPT,Consejeria de desarrollo sostenible,Direccion general de transicion energetica,True,30-1-Evaluación de Solicitud,2022-10-27 08:04:51,30,884.0,3,False
45879,1127895,1787973,<NA>,2022-02-09 09:07:17,2021-12-23,13,064,13064,Poblete,Ciudad Real,...,SLPT,Consejeria de desarrollo sostenible,Direccion general de transicion energetica,True,42-1-Propuesta Resolución Estimatoria de Conce...,2022-10-27 08:14:55,42,884.0,4,False


In [4]:
columns_estados = ['id_exp','num_tramite','fecha_tramite']
df_estados_proc_X = df_tramites_proc_X[columns_estados]
df_estados_proc_X.head()

,id_exp,num_tramite,fecha_tramite
45875,1127895,0,2021-12-23 00:00:00
45876,1127895,1,2022-02-09 09:07:17
45877,1127895,10,2022-10-14 09:44:23
45878,1127895,30,2022-10-27 08:04:51
45879,1127895,42,2022-10-27 08:14:55


In [5]:
#asignamos fecha como nueva columna, quitándonos la hora
df_estados_proc_X = df_estados_proc_X.assign(fecha=df_estados_proc_X['fecha_tramite'].dt.date)
df_estados_proc_X.drop('fecha_tramite', axis = 1, inplace = True)
df_estados_proc_X.head()
# Here are the possible ways to add a new column to a DataFrame that is calculated from another original column of the same DataFrame:

# Assign: df['new_column'] = df['original_column'] * 2
# Apply: df['new_column'] = df.apply(lambda x: x['original_column'] * 2, axis=1)
# Map: df['new_column'] = df['original_column'].map(lambda x: x * 2)
# Assign with conditional: df['new_column'] = np.where(df['original_column'] > 0, df['original_column'] * 2, 0)
# Using pandas' built-in functions: df['new_column'] = df['original_column'].rolling(window=3).mean()
# Using pandas' groupby: df.groupby('group_column')['original_column'].sum().reset_index(name='new_column')

,id_exp,num_tramite,fecha
45875,1127895,0,2021-12-23
45876,1127895,1,2022-02-09
45877,1127895,10,2022-10-14
45878,1127895,30,2022-10-27
45879,1127895,42,2022-10-27


# Número de expedientes en estado x por fecha, forward fill

## duplicados

In [6]:
duplicates = df_estados_proc_X[df_estados_proc_X.duplicated(subset=['id_exp', 'fecha', 'num_tramite'], keep=False)]
print(duplicates)

          id_exp  num_tramite       fecha
731972   1256497           30  2024-07-18
731973   1256497           42  2024-07-18
731975   1256497           30  2024-07-18
731976   1256497           42  2024-07-18
6496786  1239794           50  2024-07-18
6496787  1239794           50  2024-07-18
7234997  1171327           51  2023-10-23
7234998  1171327           51  2023-10-23
7756529  1164657           50  2023-12-01
7756532  1164657           50  2023-12-01


In [7]:
df_tramites_proc_X.loc[duplicates.index]

,id_exp,dni,nif,fecha_alta_exp,fecha_registro_exp,codine_provincia,codine_municipio,codine,municipio,provincia,...,siaci,consejeria_proc,org_instructor_proc,es_telematica,desc_tramite,fecha_tramite,num_tramite,cod_procedimiento,orden_tramite,es_inicial
731972,1256497,1851872,<NA>,2022-07-21 09:23:48,2022-07-20,45,053,45053,Consuegra,Toledo,...,SLPT,Consejeria de desarrollo sostenible,Direccion general de transicion energetica,True,30-1-Evaluación de Solicitud,2024-07-18 08:12:30,30,884.0,3,False
731973,1256497,1851872,<NA>,2022-07-21 09:23:48,2022-07-20,45,053,45053,Consuegra,Toledo,...,SLPT,Consejeria de desarrollo sostenible,Direccion general de transicion energetica,True,42-1-Propuesta Resolución Estimatoria de Conce...,2024-07-18 08:19:57,42,884.0,4,False
731975,1256497,1851872,<NA>,2022-07-21 09:23:48,2022-07-20,45,053,45053,Consuegra,Toledo,...,SLPT,Consejeria de desarrollo sostenible,Direccion general de transicion energetica,True,30-1-Evaluación de Solicitud,2024-07-18 08:42:08,30,884.0,6,False
731976,1256497,1851872,<NA>,2022-07-21 09:23:48,2022-07-20,45,053,45053,Consuegra,Toledo,...,SLPT,Consejeria de desarrollo sostenible,Direccion general de transicion energetica,True,42-1-Propuesta Resolución Estimatoria de Conce...,2024-07-18 08:42:36,42,884.0,7,False
6496786,1239794,509479,<NA>,2022-06-24 09:38:13,2022-06-22,13,034,13034,Ciudad Real,Ciudad Real,...,SLPT,Consejeria de desarrollo sostenible,Direccion general de transicion energetica,True,50-1-Resolución de Concesión,2024-07-18 07:11:53,50,884.0,5,False
6496787,1239794,509479,<NA>,2022-06-24 09:38:13,2022-06-22,13,034,13034,Ciudad Real,Ciudad Real,...,SLPT,Consejeria de desarrollo sostenible,Direccion general de transicion energetica,True,50-1-Resolución de Concesión,2024-07-18 07:11:53,50,884.0,6,False
7234997,1171327,1809681,<NA>,2022-02-21 14:18:28,2022-02-05,13,015,13015,Almodóvar del Campo,Ciudad Real,...,SLPT,Consejeria de desarrollo sostenible,Direccion general de transicion energetica,True,51-1-Resolución desestimatoria - Incumple Requ...,2023-10-23 13:03:36,51,884.0,4,False
7234998,1171327,1809681,<NA>,2022-02-21 14:18:28,2022-02-05,13,015,13015,Almodóvar del Campo,Ciudad Real,...,SLPT,Consejeria de desarrollo sostenible,Direccion general de transicion energetica,True,51-1-Resolución desestimatoria - Incumple Requ...,2023-10-23 13:03:36,51,884.0,5,False
7756529,1164657,1756793,<NA>,2022-02-10 11:52:26,2021-12-27,45,084,45084,Lillo,Toledo,...,SLPT,Consejeria de desarrollo sostenible,Direccion general de transicion energetica,True,50-1-Resolución de Concesión,2023-12-01 14:03:53,50,884.0,6,False
7756532,1164657,1756793,<NA>,2022-02-10 11:52:26,2021-12-27,45,084,45084,Lillo,Toledo,...,SLPT,Consejeria de desarrollo sostenible,Direccion general de transicion energetica,True,50-1-Resolución de Concesión,2023-12-01 14:07:26,50,884.0,9,False


In [8]:
len(df_estados_proc_X)

96479

In [9]:
df_estados_proc_X = df_estados_proc_X.drop_duplicates(
    subset=['id_exp', 'fecha', 'num_tramite'], 
    keep='last'
)

In [10]:
len(df_estados_proc_X)

96474

## fill forward

In [11]:
# df_estados_proc_X = df_estados_proc_X.sort_values(by=['id_exp', 'fecha'])
# df_estados_proc_X = df_estados_proc_X.set_index(['id_exp','fecha'])
# df_estados_proc_X

# puede haber varios cambios de estado es 1 día, hay que hacer una lista para cada expediente-dia de todos los estados por los que ha pasado
df_estados_proc_X = (
    df_estados_proc_X.reset_index()  # Reset index to access columns
    .sort_values(['id_exp', 'fecha'])  # Ensure proper sorting
    .groupby(['id_exp', 'fecha'])['num_tramite']
    .agg(list)  # Combine states into a list
    .reset_index()
    .set_index(['id_exp', 'fecha'])  # Restore MultiIndex
)
df_estados_proc_X

num_tramite
id_exp  fecha                 
1083865 2021-12-22         [0]
        2022-02-04         [1]
        2022-04-19        [10]
        2022-05-04        [30]
        2022-06-08        [42]
...                        ...
1861916 2024-03-06         [1]
1956447 2022-05-02         [0]
        2024-06-21     [1, 10]
        2024-06-26    [30, 42]
        2024-07-18        [50]

[82141 rows x 1 columns]

In [12]:
timeline = pd.date_range(df_estados_proc_X.index.get_level_values('fecha').min(),
                         df_estados_proc_X.index.get_level_values('fecha').max())

In [13]:
# Create a MultiIndex with all combinations of id_exp and timeline
multi_index = pd.MultiIndex.from_product(
    [df_estados_proc_X.index.levels[0], timeline], names=['id_exp', 'fecha']
)
multi_index

MultiIndex([(1083865, '2021-12-22'),
            (1083865, '2021-12-23'),
            (1083865, '2021-12-24'),
            (1083865, '2021-12-25'),
            (1083865, '2021-12-26'),
            (1083865, '2021-12-27'),
            (1083865, '2021-12-28'),
            (1083865, '2021-12-29'),
            (1083865, '2021-12-30'),
            (1083865, '2021-12-31'),
            ...
            (1956447, '2024-08-17'),
            (1956447, '2024-08-18'),
            (1956447, '2024-08-19'),
            (1956447, '2024-08-20'),
            (1956447, '2024-08-21'),
            (1956447, '2024-08-22'),
            (1956447, '2024-08-23'),
            (1956447, '2024-08-24'),
            (1956447, '2024-08-25'),
            (1956447, '2024-08-26')],
           names=['id_exp', 'fecha'], length=25302255)

In [14]:
print(df_estados_proc_X.index.is_unique)

True


In [15]:
# Reindex to this complete MultiIndex
df_estados_proc_X_reindexed = df_estados_proc_X.reindex(multi_index)
df_estados_proc_X_reindexed

# 1. Aligning Data to a New Index
# reindex allows you to change the index or column labels of a DataFrame or Series to match a new index or set of labels. 
# This is particularly useful for aligning data across datasets.

# import pandas as pd

# data = {'A': [1, 2, 3]}
# df = pd.DataFrame(data, index=['a', 'b', 'c'])

# # Aligning to a new index
# new_index = ['a', 'b', 'd', 'e']
# df_reindexed = df.reindex(new_index)

# print(df_reindexed)

#      A
# a  1.0
# b  2.0
# d  NaN
# e  NaN


# 2. Interpolating Data Over a Time Index
# In time-series data, reindex is often used to fill gaps or align data to a regular frequency.
# import pandas as pd
# date_range = pd.date_range('2023-01-01', '2023-01-05')
# data = {'value': [10, 15, 20]}
# df = pd.DataFrame(data, index=pd.to_datetime(['2023-01-01', '2023-01-03', '2023-01-05']))

# # Reindex to a daily frequency
# df_reindexed = df.reindex(date_range).interpolate()

# print(df_reindexed)

#            value
# 2023-01-01   10.0
# 2023-01-02   12.5
# 2023-01-03   15.0
# 2023-01-04   17.5
# 2023-01-05   20.0

num_tramite
id_exp  fecha                 
1083865 2021-12-22         [0]
        2021-12-23         NaN
        2021-12-24         NaN
        2021-12-25         NaN
        2021-12-26         NaN
...                        ...
1956447 2024-08-22         NaN
        2024-08-23         NaN
        2024-08-24         NaN
        2024-08-25         NaN
        2024-08-26         NaN

[25302255 rows x 1 columns]

In [16]:
# df_estados_proc_X_reindexed['num_tramite'] = (
#     df_estados_proc_X_reindexed['num_tramite']
#     .groupby(level=0)  # Group by `id_exp`
#     .apply(lambda group: group.ffill())  # Forward-fill lists
# )
df_estados_proc_X_reindexed['num_tramite'] = df_estados_proc_X_reindexed['num_tramite'].groupby(level=0).ffill()
df_estados_proc_X_reindexed

num_tramite
id_exp  fecha                 
1083865 2021-12-22         [0]
        2021-12-23         [0]
        2021-12-24         [0]
        2021-12-25         [0]
        2021-12-26         [0]
...                        ...
1956447 2024-08-22        [50]
        2024-08-23        [50]
        2024-08-24        [50]
        2024-08-25        [50]
        2024-08-26        [50]

[25302255 rows x 1 columns]

In [17]:
df_estados_proc_X_exploded = df_estados_proc_X_reindexed.explode('num_tramite')

In [18]:
df_estados_proc_X_exploded

num_tramite
id_exp  fecha                 
1083865 2021-12-22           0
        2021-12-23           0
        2021-12-24           0
        2021-12-25           0
        2021-12-26           0
...                        ...
1956447 2024-08-22          50
        2024-08-23          50
        2024-08-24          50
        2024-08-25          50
        2024-08-26          50

[27653290 rows x 1 columns]

In [19]:
result = (
    df_estados_proc_X_exploded.reset_index()
    .groupby(['fecha', 'num_tramite'])
    .size()
    .unstack(fill_value=0)
)
print(result)

num_tramite   0      1    10   21    30   41   42    50   51   52   ...   70   \
fecha                                                               ...         
2021-12-22    317      0    0    0     0    0    0     0    0    0  ...     0   
2021-12-23    599      0    0    0     0    0    0     0    0    0  ...     0   
2021-12-24    723      0    0    0     0    0    0     0    0    0  ...     0   
2021-12-25    745      0    0    0     0    0    0     0    0    0  ...     0   
2021-12-26    797      0    0    0     0    0    0     0    0    0  ...     0   
...           ...    ...  ...  ...   ...  ...  ...   ...  ...  ...  ...   ...   
2024-08-22   2434  15200  514    0   961   21  895  3936  403  383  ...  3257   
2024-08-23   2429  15179  510    0   989   21  920  3892  403  384  ...  3295   
2024-08-24   2429  15179  510    0   989   21  920  3892  403  384  ...  3295   
2024-08-25   2429  15179  510    0   989   21  920  3892  403  384  ...  3295   
2024-08-26   2425  15152  50

# Número de cambios por dia


In [20]:
# tabla con el número total de cambios de estado , cada día, una columna por estado
df_tramitesX_estados = df_tramitesX_estados.sort_values(by='fecha')
# no hay valores para todas las fechas, habría que rellenar las fechas vacías.
df_pivot_estados = df_tramitesX_estados.pivot_table(index='fecha', columns='num_tramite', aggfunc='size', fill_value=0)
df_pivot_estados.tail()

NameError: name 'df_tramitesX_estados' is not defined